# **Final Project: Khai thác và trực quan hóa dữ liệu**

### **Danh sách các thành viên**

<center>

|         Họ tên         |   MSSV   |
| :--------------------- | :------: |
| Trần Nguyễn Nhật Cường | 22127048 |
|    Nguyễn Công Tuấn    | 22127436 |
|     Trần Đăng Tuấn     | 22127438 |
| ... | ... |
| ... | ... |

</center>

## **Cài đặt và sử dụng các thư viện cần thiết**

In [128]:
import numpy as np
import pandas as pd
import requests

**`matplotlib`** được dùng để vẽ biểu đồ trong Python. Có thể dùng để vẽ các loại biểu đồ như
histogram, biểu đồ tán xạ (scatter plot), biểu đồ đường (line chart) và biểu đồ cột (bar chart),...

**`pandas`** được dùng để xử lý và phân tích dữ liệu dạng bảng và cung cấp cấu trúc dữ liệu như
DataFrame và Series để dễ thao tác.

**`seaborn`** dùng để vẽ biểu đồ dựa trên thư viện matplotlib nhưng biểu đồ sẽ trông đẹp hơn và cũng hỗ trợ các loại biểu đồ thống kê như violin plot, box plot, heatmap.

**`numpy`** được dùng để xử lý các phép toán số học nhanh trên mảng nhiều chiều (arrays) và hỗ trợ các phép toán ma trận, thống kê, đại số tuyến tính cần thiết cho phân tích dữ liệu.


**`requests`** được dùng để gửi các yêu cầu HTTP (như GET, POST) nhằm truy vấn dữ liệu từ các API hoặc tải dữ liệu từ Internet về máy tính.

## **Thu thập dữ liệu**

In [129]:
offset = 0
limit = 1000
all_records = []

while True:
    response = requests.get(
        url='https://data.vietnam.opendevelopmentmekong.net/en/api/3/action/datastore_search',
        params={
            'resource_id': 'a03fb5c6-4622-4196-a926-15e9542c4ae8',
            'limit': limit,
            'offset': offset
        }
    )
    
    records = response.json()['result']['records']
    if not records:
        break
    all_records.extend(records)
    offset += limit

df = pd.DataFrame(all_records)
df.head()

,_id,ID,Latitude,Longtitude,Province,District,Commune,Sub-regional,Plot,Lot,Site,Forest owner,Land type,Original resources of forest,Date,Priority,Contractor,Status
0,1,1,11.5169,108.272668,Lam Dong,Di Linh,Tam Bo,678,3.0,26.0,Tam Bo,Trần Ngọc Thế,Medium coniferous natural forest,Natural forest,2018-01-01T00:00:00,High,,Verified
1,2,2,11.4464,108.062816,Lam Dong,Di Linh,Gung Re,695,5.0,1.0,Gung Re village,Forestry One Member Limited Liability Company ...,Medium coniferous natural forest,Natural forest,2018-01-01T00:00:00,High,K' BRim (Gaoh),Verified
2,3,3,11.4462,108.062886,Lam Dong,Di Linh,Gung Re,695,5.0,3.0,Gung Re village,Forestry One Member Limited Liability Company ...,Natural wood forest in the mountains with medi...,Natural forest,2018-01-01T00:00:00,High,K' BRim (Gaoh),Verified
3,4,4,11.8010,107.937944,Lam Dong,Di Linh,Dinh Trang Thuong,607,3.0,8.0,Dinh Trang Thuong,Board of protection forest management Tan Thuong,Empty land with regenerated wood trees,Natural forest,2018-01-13T00:00:00,Low,,Not Verified
4,5,5,11.4814,108.010331,Lam Dong,Di Linh,Lien Dam,660B,7.0,43.0,Lien Dam,Group Joint Stock Company Tan Mai,Other plantation forest,Plantation forest,2018-01-13T00:00:00,Low,,Not Verified


## **Khám phá dữ liệu**

<div align='center'>

| Cột | Mô tả | 
| --- | --- |
| `_id` | 	ID tự động của hệ thống để định danh từng hàng trong cơ sở dữ liệu |
| `ID` | Mã số định danh riêng của từng khu đất/lô đất |
| `Latitude` | 	Vĩ độ (tọa độ địa lý) của khu đất/lô đất |
| `Longtitude` | Kinh độ (tọa độ địa lý) của khu đất/lô đất. |
| `Province` | Tên tỉnh/thành phố nơi khu đất/lô đất tọa lạc. |
| `District` | Tên quận/huyện nơi khu đất/lô đất tọa lạc. |
| `Commune` | Tên xã/phường/thị trấn nơi khu đất/lô đất tọa lạc. |
| `Sub-regional` | 	Khu vực/tiểu vùng nhỏ hơn trong xã hoặc huyện (thường là cụm nhỏ hơn). |
| `Plot` | Số hiệu thửa đất (Plot number). |
| `Lot` | Số hiệu lô đất trong thửa đất. |
| `Site` | Tên cụ thể của khu vực/quần thể đất (ví dụ: tên thôn, làng). |
| `Forest owner	` | Tên chủ sở hữu hoặc đơn vị quản lý rừng. |
| `Land type` | Loại hình đất tại khu vực (ví dụ: rừng tự nhiên trung bình, đất rừng trồng...). |
| `Original resources of forest` | Loại tài nguyên rừng ban đầu (ví dụ: rừng tự nhiên, rừng trồng...). |
| `Date` | Ngày ghi nhận hoặc cập nhật thông tin khu đất/lô đất. |
| `Priority` | Mức độ ưu tiên quản lý hoặc bảo vệ khu đất/lô đất (ví dụ: High - Cao). | 
| `Contractor` | Tên nhà thầu hoặc cá nhân phụ trách khai thác hoặc trồng rừng (nếu có). |
| `Status` | Tình trạng hiện tại của khu đất/lô đất (ví dụ: Verified - Đã xác minh). |

</div>

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19545 entries, 0 to 19544
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _id                           19545 non-null  int64  
 1   ID                            19545 non-null  int64  
 2   Latitude                      19545 non-null  float64
 3   Longtitude                    19545 non-null  float64
 4   Province                      19545 non-null  object 
 5   District                      19545 non-null  object 
 6   Commune                       19545 non-null  object 
 7   Sub-regional                  19545 non-null  object 
 8   Plot                          19304 non-null  float64
 9   Lot                           18456 non-null  float64
 10  Site                          19545 non-null  object 
 11  Forest owner                  19545 non-null  object 
 12  Land type                     19545 non-null  object 
 13  O

## **Tiền xử lý dữ liệu**

In [131]:
df.replace(to_replace='', value=np.nan, inplace=True)
round(df.isna().sum() / df.shape[0] * 100, 2)

_id                              0.00
ID                               0.00
Latitude                         0.00
Longtitude                       0.00
Province                         0.00
District                         0.00
Commune                          0.01
Sub-regional                     0.01
Plot                             1.23
Lot                              5.57
Site                             1.00
Forest owner                    25.60
Land type                        0.13
Original resources of forest     0.00
Date                             0.01
Priority                         0.00
Contractor                      98.75
Status                           0.00
dtype: float64

In [132]:
df.drop(columns=['_id', 'ID', 'Contractor'], inplace=True)
df.head()

,Latitude,Longtitude,Province,District,Commune,Sub-regional,Plot,Lot,Site,Forest owner,Land type,Original resources of forest,Date,Priority,Status
0,11.5169,108.272668,Lam Dong,Di Linh,Tam Bo,678,3.0,26.0,Tam Bo,Trần Ngọc Thế,Medium coniferous natural forest,Natural forest,2018-01-01T00:00:00,High,Verified
1,11.4464,108.062816,Lam Dong,Di Linh,Gung Re,695,5.0,1.0,Gung Re village,Forestry One Member Limited Liability Company ...,Medium coniferous natural forest,Natural forest,2018-01-01T00:00:00,High,Verified
2,11.4462,108.062886,Lam Dong,Di Linh,Gung Re,695,5.0,3.0,Gung Re village,Forestry One Member Limited Liability Company ...,Natural wood forest in the mountains with medi...,Natural forest,2018-01-01T00:00:00,High,Verified
3,11.8010,107.937944,Lam Dong,Di Linh,Dinh Trang Thuong,607,3.0,8.0,Dinh Trang Thuong,Board of protection forest management Tan Thuong,Empty land with regenerated wood trees,Natural forest,2018-01-13T00:00:00,Low,Not Verified
4,11.4814,108.010331,Lam Dong,Di Linh,Lien Dam,660B,7.0,43.0,Lien Dam,Group Joint Stock Company Tan Mai,Other plantation forest,Plantation forest,2018-01-13T00:00:00,Low,Not Verified


In [133]:
df.duplicated().sum()

np.int64(693)

In [134]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

np.int64(0)

In [135]:
df['Date'] = pd.to_datetime(arg=df['Date'], format='%Y-%m-%dT%H:%M:%S')
df['Date'].head()

0   2018-01-01
1   2018-01-01
2   2018-01-01
3   2018-01-13
4   2018-01-13
Name: Date, dtype: datetime64[ns]

In [136]:
for col in df.select_dtypes(include='object').columns:
    print(f'{col}: {df[col].unique()}')

Province: ['Lam Dong' 'Nghe An']
District: ['Di Linh' 'Tuong Duong' 'Lac Duong' 'Ky Son']
Commune: ['Tam Bo' 'Gung Re' 'Dinh Trang Thuong' 'Lien Dam' 'Gia Bac' 'Bao Thuan'
 'Son Dien' 'Gia Hiep' 'Hoa Bac' 'Tan Chau' 'Tan Lam' 'Hoa Trung'
 'Dinh Lac' 'Tan Thuong' 'Tam Quang' 'Tam Dinh' "Dung KN'o" 'Lat' 'Da Sar'
 'Lac Duong Town' 'Da Chais' 'Da Nhim' 'Tam Thai' 'Huu Kiem' 'Tay Son'
 'Na Ngoi' 'Pha Danh' 'Ta Ca' 'Chieu Luu' 'Muong Tip' 'Thach Giam'
 'Yen Thang' 'Huu Lap' 'Tam Hop' 'Nam Can' nan]
Sub-regional: ['678' '695' '607' '660B' '736' '709A' '735' '674' '716' '743' '719'
 '686A' '679' '682' '701' '722' '612' '636A' '666' '675' '606' '609'
 '660A' '663' '687A' '709B' '697' '696' '692' '715' '733' '717' '734'
 '738' '698' '702' '621B' '645' '680' '727' '623' '671' '693' '711' '659A'
 '732' '673' '659B' '708' '647A' '677' '704' '725' '731' '737A' '661A'
 '706' '724' '694' '726' '707A' '712' '723' '742' '739' '688' '690' '665'
 '686B' '676' '721' '618B' '618A' '664' '672' '662B' '713' 

**Nhận xét**
- `Nam Can` có thể là xã **Nậm Càn** hoặc **Nậm Cắn**, cần dựa vào vĩ độ và kinh độ để xác định chính xác đó là xã nào.
- `Lac Duong Town` có thê đổi tên thành `Lac Duong` vì đã chia theo tỉnh/huyện/xã nên có thể dễ dàng **tỉnh Lạc Dương** và **thị trấn Lạc Dương**.
- `Dung KN'o` bị dư dấu `'` nên sẽ được loại bỏ.

In [137]:
df[df['Commune'] == 'Nam Can']

,Latitude,Longtitude,Province,District,Commune,Sub-regional,Plot,Lot,Site,Forest owner,Land type,Original resources of forest,Date,Priority,Status
17657,19.234808,104.262505,Nghe An,Ky Son,Nam Can,497,2.0,2.0,Nam Can commune,Board of protection forest management Ky Son,Poor evergreen broad-leaved natural forest on ...,Natural forest,2022-03-23,Medium,Not Verified
17772,19.234080,104.219074,Nghe An,Ky Son,Nam Can,496,4.0,2.0,Nam Can commune,Board of protection forest management Ky Son,Mixed rich natural forest on the soil mountain,Natural forest,2022-05-10,Medium,Not Verified
18336,19.234718,104.262506,Nghe An,Ky Son,Nam Can,497,2.0,2.0,Nam Can commune,Board of protection forest management Ky Son,Poor evergreen broad-leaved natural forest on ...,Natural forest,2022-06-15,Medium,Not Verified


**Nhận xét**
- Dựa vào **vĩ độ** và **kinh độ**, nhóm xác định được cả 3 vị trí này đều thuộc xã **Nam Càn**, huyện **Kỳ Sơn**, tỉnh **Nghệ An**.

In [138]:
df['Commune'] = np.where(
    df['Commune'] == 'Lac Duong Town', 'Lac Duong', np.where(
        df['Commune'] == "Dung KN'o", 'Dung KNo', df['Commune']
    )
)
df['Commune'].unique()

array(['Tam Bo', 'Gung Re', 'Dinh Trang Thuong', 'Lien Dam', 'Gia Bac',
       'Bao Thuan', 'Son Dien', 'Gia Hiep', 'Hoa Bac', 'Tan Chau',
       'Tan Lam', 'Hoa Trung', 'Dinh Lac', 'Tan Thuong', 'Tam Quang',
       'Tam Dinh', 'Dung KNo', 'Lat', 'Da Sar', 'Lac Duong', 'Da Chais',
       'Da Nhim', 'Tam Thai', 'Huu Kiem', 'Tay Son', 'Na Ngoi',
       'Pha Danh', 'Ta Ca', 'Chieu Luu', 'Muong Tip', 'Thach Giam',
       'Yen Thang', 'Huu Lap', 'Tam Hop', 'Nam Can', nan], dtype=object)

**Tổng kết các tỉnh/huyện/xã**
- **Lâm Đồng (20)**:
    - **Di Linh (14)**: Bảo Thuận, Đinh Lạc, Đinh Trang Thượng, Gia Bắc, Gia Hiệp, Gung Ré, Hòa Bắc, Hòa Trung, Liên Đầm, Sơn Điền, Tam Bố, Tân Châu, Tân Lâm, Tân Thượng.
    - **Lạc Dương (6)**: Thị trấn Lạc Dương, Đạ Chais, Đạ Nhim, Đạ Sar, Đưng KNớ, Lát.

- **Nghệ An (15)**:
    - **Tương Dương (6)**: Thị trấn Thạch Giám, Tam Đinh, Tam Hợp, Tam Quang, Tam Thái, Yên Thắng.
    - **Kỳ Sơn (9)**: Chiêu Lưu, Hữu Kiệm, Hữu Lập, Mường Típ, Na Ngoi, Nậm Càn, Pha Danh, Tà Cạ, Tây Sơn.